In [1]:
import os
import glob
import re

from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def string2geocoord(string):
    coord = re.sub('S|N|E|W', '.', string)    
    if 'S' in string or 'W' in string:
        return '-' + coord
    else:
        return coord    

    
def is_company_data(table):
    return tb.find('label').get('id') == 'labelNúmero/Nome'


def is_erb_data(table):
    return tb.find('label').text == 'Estação'

In [3]:
DATA_DIR = '../data'
DATA_FILES = glob.glob(os.path.join(DATA_DIR, '*.html'))
print(f'Encontrados {len(DATA_FILES)} arquivos de dados.')

Encontrados 27 arquivos de dados.


In [4]:
dados_br = pd.DataFrame()
converters = {'Estação': str}
for n, datafile in enumerate(DATA_FILES):
    print(n, '-', datafile)
    data = {}
    soup = BeautifulSoup(open(datafile, encoding='Latin1').read(), 'lxml')
    tables = soup.find_all('table')
    for tb in tables:
        if is_company_data(tb):
            company_id = tb.find('label').text        
            company_number = company_id[:7].strip()
            company_name = company_id[10:].strip()
            company_sgl = company_name.split(' ')[0]        
        else:
            if is_erb_data(tb):
                HTML = str(tb)
                df = pd.read_html(HTML, header=0, converters=converters)[0]
                df.insert(0, 'nom_operadora', company_name)
                df.insert(0, 'sgl_operadora', company_sgl)
                df.insert(0, 'num_operadora', company_number)            
                data[company_name] = df
    dados_br = pd.concat([dados_br, pd.concat([data[i] for i in data], ignore_index=True)], ignore_index=True)

0 - ../data\AC.html
1 - ../data\AL.html
2 - ../data\AM.html
3 - ../data\AP.html
4 - ../data\BA.html
5 - ../data\CE.html
6 - ../data\DF.html
7 - ../data\ES.html
8 - ../data\GO.html
9 - ../data\MA.html
10 - ../data\MG.html
11 - ../data\MS.html
12 - ../data\MT.html
13 - ../data\PA.html
14 - ../data\PB.html
15 - ../data\PE.html
16 - ../data\PI.html
17 - ../data\PR.html
18 - ../data\RJ.html
19 - ../data\RN.html
20 - ../data\RO.html
21 - ../data\RR.html
22 - ../data\RS.html
23 - ../data\SC.html
24 - ../data\SE.html
25 - ../data\SP.html
26 - ../data\TO.html


In [5]:
dados_br.head()

num_operadora sgl_operadora                 nom_operadora    Estação  \
0       4152433        NEXTEL  NEXTEL TELECOMUNICACOES LTDA  698421370   
1       4152433        NEXTEL  NEXTEL TELECOMUNICACOES LTDA  699136172   
2       4152433        NEXTEL  NEXTEL TELECOMUNICACOES LTDA  699136237   
3       4152433        NEXTEL  NEXTEL TELECOMUNICACOES LTDA  699521963   
4       4152433        NEXTEL  NEXTEL TELECOMUNICACOES LTDA  699521980   

          Nome  UF   Município            Bairro  \
0  ACCRBO009OU  AC  Rio Branco         Vila Nova   
1  ACCRBO004OU  AC  Rio Branco  Conjunto Tangará   
2  ACCRBO013OU  AC  Rio Branco          Alumínio   
3  ACCRBO001OU  AC  Rio Branco       Barro Preto   
4  ACCRBO002OU  AC  Rio Branco            Centro   

                                          Logradouro   Latitude  Longitude  \
0            Avenida Antônio da Rocha Viana - nº s/n  09S561476  67W493986   
1               Avenida Nações Unidas, s/n° - nº S/N  09S575120  67W502730   
2                   Rua 05 Quadra 7 Lote 17 - nº S/N  09S571380  67W500270   
3  Travessa Santa Tereza esquina com Invernada - ...  09S580090  67W473690   
4                Avenida Francisco Pinheiro - nº s/n  10S005260  67W465870   

  Data Cadastro Data 1º Lic. Última Licença  
0    10/12/2012   09/01/2013     09/06/2014  
1    01/07/2013   12/09/2013     09/06/2014  
2    01/07/2013   12/09/2013     09/06/2014  
3    25/10/2013   13/02/2014     09/06/2014  
4    25/10/2013   13/02/2014     09/06/2014

In [6]:
dados_br.Latitude = dados_br.Latitude.map(string2geocoord)
dados_br.Longitude = dados_br.Longitude.map(string2geocoord)

In [7]:
dados_br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86243 entries, 0 to 86242
Data columns (total 14 columns):
num_operadora     86243 non-null object
sgl_operadora     86243 non-null object
nom_operadora     86243 non-null object
Estação           86243 non-null object
Nome              86154 non-null object
UF                86243 non-null object
Município         86243 non-null object
Bairro            84656 non-null object
Logradouro        86243 non-null object
Latitude          86243 non-null object
Longitude         86243 non-null object
Data Cadastro     86243 non-null object
Data 1º Lic.      84241 non-null object
Última Licença    80080 non-null object
dtypes: object(14)
memory usage: 9.2+ MB


In [9]:
dados_br.groupby(['UF', 'sgl_operadora']).Estação.count()

UF  sgl_operadora
AC  CLARO              55
    NEXTEL             35
    OI                 52
    TELEFÔNICA         82
    TIM                46
AL  CLARO             193
    NEXTEL             41
    OI                234
    TELEFÔNICA        275
    TIM               299
AM  CLARO             192
    NEXTEL            152
    OI                149
    TELEFÔNICA        514
    TIM               236
AP  CLARO              33
    NEXTEL             23
    OI                 65
    TELEFÔNICA         78
    TIM                50
BA  CLARO            1153
    NEXTEL            414
    OI                730
    TELEFÔNICA        870
    TIM               943
CE  CLARO             486
    NEXTEL            197
    OI                506
    TELEFÔNICA        944
    TIM               738
                     ... 
RR  OI                 15
    TELEFÔNICA         64
    TIM                34
RS  CLARO            1173
    NEXTEL            349
    OI               1306
    TELEFÔNICA      

In [10]:
dados_br.to_csv('../data/dataset-erb-BR.csv', sep=';', encoding='Latin1', index=False)